In [1]:
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd



In [ ]:
test = np.loadtxt("mnist_test.csv", delimiter = ",")
train = np.loadtxt("mnist_train.csv", delimiter = ",")

In [ ]:
train_X, train_Y = train[:,1:].reshape(60000, 28 * 28 ), train[:,0]
test_X, test_Y = test[:,1:].reshape(10000, 28 * 28 ), test[:,0]

train_X.shape, train_Y.shape, test_X.shape, test_Y.shape



In [ ]:
model = svm.SVC(C=20)
model.fit(train_X, train_Y)

In [ ]:
preds = model.predict(test_X)

In [ ]:
accuracy_score(preds, test_Y)

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision.transforms import ToTensor


In [ ]:
transform = transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])

trainset = datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)



In [ ]:
testset = datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)

In [ ]:
trainset

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 128, 3, 1)
        self.conv3 = nn.Conv2d(128, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(5 * 5 * 64, 128)
        self.fc2 = nn.Linear(128, 32)
        self.fc2 = nn.Linear(32, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output
    
    

In [ ]:
net = Net()

In [3]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-4)


NameError: name 'nn' is not defined

In [2]:
running_loss = 0
for epochs in range(2):
    for i, data in enumerate(trainloader):
        inputs, labels = data
        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 0:  
            print('[%d %5d] loss: %.3f' %
                  (epochs, i + 1, running_loss / 100))
            running_loss = 0.0

NameError: name 'trainloader' is not defined

In [ ]:
num_true= 0
with torch.no_grad():
    for data in testloader:
        images,labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        num_true += len(np.argwhere((predicted == labels).numpy() == True))
        print(len(np.argwhere((predicted == labels).numpy() == True)))
        


In [ ]:
print("accuracy: ",num_true / 10000)